# Analyse Climatique : Évolution de la Température Moyenne Cumulative par Année

L'objectif de cette analyse est d'explorer les données météorologiques quotidiennes relevées dans différentes stations en France pour calculer la somme cumulative de la température moyenne journalière pour chaque année. Les données utilisées proviennent du fichier `Q_34_latest-2024-2025_RR-T-Vent.csv.gz` disponible sur le site [data.gouv.fr](https://object.files.data.gouv.fr/meteofrance/data/synchro_ftp/BASE/QUOT/Q_34_latest-2024-2025_RR-T-Vent.csv.gz). Ce fichier contient des relevés météorologiques quotidiens, notamment les températures minimales, maximales et moyennes, ainsi que les précipitations et la vitesse du vent.

L'analyse se concentre sur la température moyenne journalière et vise à moyenner ces valeurs sur toutes les stations pour chaque jour de l'année, puis à calculer la somme cumulative de ces moyennes pour obtenir une représentation de l'évolution de la température moyenne cumulative par année. Cela permettra d'identifier les tendances et les variations dans les températures moyennes au fil des ans.


## Méthodologie

La méthodologie utilisée dans cette analyse implique plusieurs étapes. Tout d'abord, les données sont chargées et traitées à l'aide de requêtes SQL pour extraire les informations nécessaires. Une première requête calcule la moyenne de la température moyenne journalière (`TM`) pour chaque jour de l'année en moyennant sur toutes les stations. Ensuite, une seconde requête calcule la somme cumulative de ces moyennes pour chaque année. Les résultats sont ensuite visualisés à l'aide de graphiques linéaires pour représenter l'évolution de la température moyenne cumulative par année.


 

Les résultats de cette analyse sont présentés sous forme de graphique linéaire interactif, permettant de visualiser l'évolution de la température moyenne cumulative pour chaque année et de comparer les tendances entre les différentes années.

## 🔧 Configuration

In [1]:
# Installation et imports
import duckdb as ddb
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

## 🦆 Chargement du dataset avec Duckdb

In [2]:
# Fonction de chargement complète (basée sur load_file_from_url_lite)
def load_file_from_url_lite(url_dataset="", loader="read_csv_auto", options="", nom_table="loaded_dataset", safe_mode=False):
    ddb.execute("install spatial")
    ddb.execute("load spatial")
    ddb.execute("INSTALL h3 FROM community")
    ddb.execute("LOAD h3")
    ddb.execute("install webbed from community;")
    ddb.execute("load webbed")
    ddb.execute("set force_download=True")
    ddb.execute(f"drop table if exists {nom_table}")   
    
    # Détection automatique du type de fichier
    if 'csv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'tsv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'txt' in url_dataset: 
        loader = "read_csv_auto"
    elif 'parquet' in url_dataset: 
        loader = "read_parquet"
    elif 'json' in url_dataset: 
        loader = "read_json_auto"
    elif 'xls' in url_dataset or 'xlsx' in url_dataset: 
        loader = "st_read"
    elif 'shp' in url_dataset: 
        loader = "st_read"
    elif 'geojson' in url_dataset: 
        loader = "st_read"
    elif 'xml' in url_dataset: 
        loader = "read_xml"
    elif 'html' in url_dataset: 
        loader = "read_html"
    else: 
        raise ValueError(f"Type de fichier non supporté pour {url_dataset}")
    
    if options=="": 
        options = "" 
    if 'csv' in url_dataset and safe_mode==True: 
        options = ", all_varchar=1" 
    if nom_table=="": 
        nom_table = "loaded_dataset"
    
    try:
        status = ddb.sql(f"""
            create or replace table {nom_table} as select *
            from
            {loader}("{url_dataset}" {options})
        """)
        return status
    except Exception as e:
        return f"Erreur au chargement du fichier : {str(e)}"

def run_query(sql):
    return ddb.sql(sql.replace("`"," ")).to_df()

# Chargement des données
load_file_from_url_lite("https://object.files.data.gouv.fr/meteofrance/data/synchro_ftp/BASE/QUOT/Q_34_latest-2024-2025_RR-T-Vent.csv.gz", safe_mode=True)
print("✅ Données chargées avec succès")

✅ Données chargées avec succès


## 🔍 Analyse SQL

Cette requête utilise des techniques SQL pour extraire et transformer les données de manière efficace.

In [3]:
# Exécution de la requête
df = run_query(""" WITH daily_avg_tm AS (
  SELECT 
    SUBSTRING(CAST("AAAAMMJJ" AS VARCHAR), 1, 4) AS year,
    SUBSTRING(CAST("AAAAMMJJ" AS VARCHAR), 5, 2) AS month,
    SUBSTRING(CAST("AAAAMMJJ" AS VARCHAR), 7, 2) AS day,
    AVG(CAST("TM" AS DOUBLE)) AS avg_tm
  FROM loaded_dataset
  GROUP BY "AAAAMMJJ"
),
cumulative_sum AS (
  SELECT 
    year, 
    month, 
    day, 
    avg_tm,
    SUM(avg_tm) OVER (PARTITION BY year ORDER BY month, day) AS cumulative_tm
  FROM daily_avg_tm
)
SELECT 
  year, 
  month, 
  day, 
  cumulative_tm
FROM cumulative_sum
ORDER BY year, month, day """)
print(f"Résultats : {len(df)} lignes")
df.head()

Résultats : 597 lignes


,year,month,day,cumulative_tm
0,2024,01,01,7.317857
1,2024,01,02,14.139286
2,2024,01,03,24.760714
3,2024,01,04,35.657143
4,2024,01,05,44.707143


## 📈 Visualisation

La bibliothèque principale utilisée pour cette visualisation de données est Plotly Express, qui est une surcouche de Plotly. Cette technologie est adaptée pour représenter des données temporelles multivariées, comme ici l'évolution de la température moyenne cumulative par année, avec une grande interactivité. Plotly Express permet une représentation claire et interactive des données avec une personnalisation avancée.

In [4]:
import pandas as pd
import duckdb as ddb
import pandas as pd
import plotly.express as px
import datetime

df['date'] = pd.to_datetime(df[['year', 'month', 'day']].astype(str).agg('-'.join, axis=1), format='%Y-%m-%d')
df['dayofyear'] = df['date'].dt.dayofyear
df['year'] = df['year'].astype(int)

dataviz = px.line(df,
                  x='dayofyear',
                  y='cumulative_tm',
                  color=df['year'].astype(str),
                  labels={'dayofyear': 'Jour de l\'année',
                          'cumulative_tm': 'Température Moyenne Cumulative (°C)',
                          'color': 'Année'},
                  title='Évolution de la Température Moyenne Cumulative par Année - Analyse Climatique',
                  hover_data={'date': True, 'cumulative_tm': ':.2f°C'},
                  color_discrete_sequence=px.colors.qualitative.Vivid)

dataviz.update_layout(height=600,
                      font=dict(family="Raleway", size=12),
                      margin=dict(l=20, r=20, t=60, b=20))
dataviz.update_xaxes(range=[1, 366], dtick=30)
dataviz

---
*Made with ❤️ and with [duckit.fr](https://duckit.fr) - [Ali Hmaou](https://www.linkedin.com/in/ali-hmaou-6b7b73146/)*

In [5]:

# --- Variables injectées par le script ---
FINAL_OBJECT_VARIABLE_NAME = 'dataviz'
OUTPUT_IMAGE_NAME = 'published\\notebooks\\duckit_analysis_20250820_211330.png'
OUTPUT_HTML_NAME = 'published\\notebooks\\duckit_analysis_20250820_211330.html'

# ===================================================================
# CELLULE INJECTÉE AUTOMATIQUEMENT (VERSION ROBUSTE)
# ===================================================================
import sys
import os
# On importe les modules nécessaires pour l'export au cas où
try:
    from bokeh.io import save as bokeh_save
except ImportError:
    bokeh_save = None

try:
    # On s'assure que le dossier de sortie existe
    output_dir = os.path.dirname(OUTPUT_IMAGE_NAME)
    if output_dir:
        os.makedirs(output_dir, exist_ok=True)

    # On utilise globals().get() pour une récupération plus sûre
    final_object = globals().get(FINAL_OBJECT_VARIABLE_NAME)

    if final_object is None:
        # On lève une NameError pour être cohérent avec le code original
        raise NameError(f"name '{FINAL_OBJECT_VARIABLE_NAME}' is not defined")

    print(f"INFO: Variable '{FINAL_OBJECT_VARIABLE_NAME}' trouvée. Tentative d'exportation...")

    object_type = str(type(final_object))

    if 'plotly.graph_objs._figure.Figure' in object_type:
        print(f"--> Détecté : Plotly. Sauvegarde HTML et PNG.")
        # 1. Sauvegarde HTML pour l'interactivité
        print(f"--> Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        final_object.write_html(OUTPUT_HTML_NAME, include_plotlyjs='cdn')
        # 2. Sauvegarde PNG pour l'aperçu statique
        try:
            print(f"--> Tentative de sauvegarde PNG directe dans : {OUTPUT_IMAGE_NAME}")
            final_object.write_image(OUTPUT_IMAGE_NAME, scale=3, width=1200, height=800)
            print(f"--> Image Plotly sauvegardée avec succès.")
        except Exception as e:
            print(f"AVERTISSEMENT: La sauvegarde directe en PNG a échoué (kaleido est-il installé?).", file=sys.stderr)
            print(f"   Erreur: {e}", file=sys.stderr)
            print(f"--> PLAN B: On va utiliser la capture d'écran du HTML à la place.")
            # On crée un fichier marqueur pour que le script de post-traitement prenne le relais
            with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
                f.write("plotly")
    elif 'folium.folium.Map' in object_type:
        print(f"--> Détecté : Folium. Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        final_object.save(OUTPUT_HTML_NAME)
        # On crée un fichier marqueur générique pour la capture d'écran
        print(f"--> Création du marqueur de capture d'écran.")
        with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
            f.write("folium")
    elif 'altair.vegalite' in object_type and hasattr(final_object, 'save'):
        print(f"--> Détecté : Altair. Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        final_object.save(OUTPUT_HTML_NAME)
        # On crée un fichier marqueur générique pour la capture d'écran
        print(f"--> Création du marqueur de capture d'écran.")
        with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
            f.write("altair")
    elif 'bokeh.plotting' in object_type and bokeh_save is not None:
        print(f"--> Détecté : Bokeh. Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        bokeh_save(final_object, filename=OUTPUT_HTML_NAME, title="")
        # On crée un fichier marqueur générique pour la capture d'écran
        print(f"--> Création du marqueur de capture d'écran.")
        with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
            f.write("bokeh")
    elif 'matplotlib.figure.Figure' in object_type:
        print(f"--> Détecté : Matplotlib. Sauvegarde dans : {OUTPUT_IMAGE_NAME}")
        final_object.savefig(OUTPUT_IMAGE_NAME, dpi=300, bbox_inches='tight')
    else:
        print(f"AVERTISSEMENT: Type non supporté : {object_type}", file=sys.stderr)
except NameError:
    print(f"AVERTISSEMENT: Aucune variable '{FINAL_OBJECT_VARIABLE_NAME}' trouvée.", file=sys.stderr)
except Exception as e:
    print(f"ERREUR lors de l'exportation : {e}", file=sys.stderr)


INFO: Variable 'dataviz' trouvée. Tentative d'exportation...
--> Détecté : Plotly. Sauvegarde HTML et PNG.
--> Sauvegarde HTML dans : published\notebooks\duckit_analysis_20250820_211330.html
--> Tentative de sauvegarde PNG directe dans : published\notebooks\duckit_analysis_20250820_211330.png


--> Image Plotly sauvegardée avec succès.
